In [1]:
import glob,os
import gzip

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
plt.switch_backend('agg')
import seaborn as sns
sns.set_style( "white" )

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager
from matplotlib import font_manager
from matplotlib.colors import LinearSegmentedColormap

font_dirs = ['/home/jiangquanlong/miniconda3/envs/r_env/lib/python3.12/site-packages/matplotlib/mpl-data/fonts/ttf']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']  
plt.rcParams['pdf.fonttype'] = 42

In [2]:
def get_sc(fname):
    df = pd.read_csv(fname,header=0)
    test_group = list(set(df['test_group']))[0]
    df = df[df['test_group']==test_group]
    df['pos'] = df['name'].str.split('_').str[1:3].str.join('_')
    df = df.sort_values(by='p_value_adj')
    df.index = df['pos']
    return df[['pos','p_value_adj']]

In [3]:
def get_rmats(fname):
    df = pd.read_table(fname, header=0)
    df['3end'] = df['chr'].astype(str) + '_' + df['downstreamES'].astype(str)
    df['SE_pos'] = df['chr'].astype(str) + '_' + df['exonStart_0base'].astype(str) + '_' + df['exonEnd'].astype(str)
    df['5end_3end'] = df['chr'].astype(str) + '_' + df['upstreamEE'].astype(str) + '_' + df['downstreamES'].astype(str)
    df = df.sort_values(by='FDR')
    return df[['3end','SE_pos','5end_3end','IncLevelDifference','FDR']]

In [4]:
def get_leaf(fname):
    pos_f = os.path.split(fname)[0] + '/leafcutter_cluster/sample_perind_numers.counts.gz'
    with gzip.open(pos_f, 'rt') as file:  
        lines = list(map(lambda l: l.strip().split(' '), file.readlines()))[1:]
    clu_dict = {}
    for line in lines:
        chrom,start,end,clu_id = line[0].split(':')
        clu_id = chrom + ':' + clu_id
        if clu_id not in clu_dict:
            clu_dict[clu_id] = [[],[]]
        clu_dict[clu_id][0].append('_'.join([chrom,start,end]))
        clu_dict[clu_id][1].append('_'.join([chrom,end]))
    
    df = pd.read_table(fname,header=0)
    pos_list = []
    three_end_list = []
    for clu_id in df['cluster']:
        poss = clu_dict[clu_id][0]; three_ends = clu_dict[clu_id][1]
        pos_list.append(poss)
        three_end_list.append(three_ends)
    df['5end_3end'] = pos_list
    df['3end'] = three_end_list
    df['5end_3end_tuple'] = df['5end_3end'].apply(tuple)
    df = df.sort_values('p.adjust').drop_duplicates(subset='5end_3end_tuple', keep='first')
    df =  df.sort_values(by='p.adjust')
    return df[['cluster','5end_3end','3end','p.adjust']]
        

In [5]:
def find_similar_intervals_rmats(list_A, list_B, tolerance=3):
    # 将list_B转换为更快查找的格式
    def parse_interval(interval):
        parts = interval.split('_')
        if len(parts) == 3:
            return parts[0], int(parts[1]), int(parts[2])
        elif len(parts) == 2:
            return parts[0], int(parts[1])
        else:
            ...
    # 创建一个字典，键是染色体，值是起始位置列表
    b_intervals_by_chrom = {}
    for interval_b in list_B:
        chrom_b, start_b = parse_interval(interval_b)
        if chrom_b not in b_intervals_by_chrom:
            b_intervals_by_chrom[chrom_b] = []
        b_intervals_by_chrom[chrom_b].append(start_b)
    
    common_intervals = []
    
    # 遍历list_A
    for interval_a in list_A:
        chrom_a, start_a,end_a = parse_interval(interval_a)
        
        # 只在同一染色体的区间中查找
        if chrom_a not in b_intervals_by_chrom:
            continue
        
        # 在同一染色体的起始位置中查找匹配
        for start_b in b_intervals_by_chrom[chrom_a]:
            if (abs(start_a - start_b) <= tolerance) or (abs(end_a - start_b) <= tolerance):
                common_intervals.append(interval_a)
                break
    
    return common_intervals

In [6]:
def find_similar_intervals_leaf(list_A, list_B, tolerance=3):
    # 将list_B转换为更快查找的格式
    def parse_interval(interval):
        parts = interval.split('_')
        if len(parts) == 3:
            return parts[0], int(parts[1]), int(parts[2])
        elif len(parts) == 2:
            return parts[0], int(parts[1])
        else:
            ...
    # 创建一个字典，键是染色体，值是起始位置列表
    b_intervals_by_chrom = {}
    for interval_b in list_B:
        chrom_b, start_b = parse_interval(interval_b)
        if chrom_b not in b_intervals_by_chrom:
            b_intervals_by_chrom[chrom_b] = []
        b_intervals_by_chrom[chrom_b].append(start_b)
    
    common_intervals = []
    
    for interval_a_list in list_A:
        found = False
        for interval_a in interval_a_list:
            chrom_a, start_a, end_a = parse_interval(interval_a)
            if chrom_a not in b_intervals_by_chrom:
                continue
            for start_b in b_intervals_by_chrom[chrom_a]:
                if abs(start_a - start_b) <= tolerance or abs(end_a - start_b) <= tolerance:
                    common_intervals.append(interval_a)
                    found = True
                    break
            if found:
                break
    return common_intervals

In [7]:
def remove_duplicates(lst):
    seen = set()
    return [x for x in lst if not (x in seen or seen.add(x))]

In [8]:
def compute_aucc(ranked_genes_bulk, ranked_genes_sc, k,tool, plot=False):
    ranked_genes_sc = remove_duplicates( ranked_genes_sc)
    intersection_counts = []
    for i in range(1, k + 1):
        top_bulk = ranked_genes_bulk[:i]
        top_sc = ranked_genes_sc[:i]
        if tool == 'rmats':
            intersection = len(find_similar_intervals_rmats(top_bulk, top_sc))
        else:
            intersection = len(find_similar_intervals_leaf(top_bulk, top_sc))
        intersection_counts.append(intersection)

    # 计算面积（Raw AUCC）
    raw_aucc = sum(intersection_counts)

    # 归一化：最大面积是 k*(k+1)/2
    max_area = k * (k + 1) / 2
    normalized_aucc = raw_aucc / max_area

    if plot:
        plt.figure(figsize=(6, 4))
        plt.plot(range(1, k + 1), intersection_counts, label='Concordance Curve', color='blue')
        plt.fill_between(range(1, k + 1), intersection_counts, alpha=0.3, color='blue')
        plt.title(f'Concordance Curve (AUCC = {normalized_aucc:.3f})')
        plt.xlabel('Top-k Genes')
        plt.ylabel('Intersection Size')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

    return normalized_aucc

In [9]:
def compute_aucc_gene(name,ranked_genes_bulk, ranked_genes_sc, k,tool, plot=False):
    def find_same_gene(list_A,list_B):
        common_genes = []
        for gene in list_A:
            if gene in list_B:
                common_genes.append(gene)
        return common_genes
        
    ranked_genes_sc = remove_duplicates(ranked_genes_sc)
    #convert to gene id
    if name == 'data9_a2i':
        ranked_genes_sc = convert_to_geneid(ranked_genes_sc[:k+1],mm_anno_df)
        ranked_genes_bulk = convert_to_geneid(ranked_genes_bulk[:k+1],mm_anno_df)
    else:
        ranked_genes_sc = convert_to_geneid(ranked_genes_sc[:k+1],hg_anno_df)
        ranked_genes_bulk = convert_to_geneid(ranked_genes_bulk[:k+1],hg_anno_df)
    intersection_counts = []
    for i in range(1, k + 1):
        top_bulk = ranked_genes_bulk[:i]
        top_sc = ranked_genes_sc[:i]
        intersection = len(find_same_gene(top_bulk, top_sc))
        intersection_counts.append(intersection)

    # 计算面积（Raw AUCC）
    raw_aucc = sum(intersection_counts)

    # 归一化：最大面积是 k*(k+1)/2
    max_area = k * (k + 1) / 2
    normalized_aucc = raw_aucc / max_area

    if plot:
        plt.figure(figsize=(6, 4))
        plt.plot(range(1, k + 1), intersection_counts, label='Concordance Curve', color='blue')
        plt.fill_between(range(1, k + 1), intersection_counts, alpha=0.3, color='blue')
        plt.title(f'Concordance Curve (AUCC = {normalized_aucc:.3f})')
        plt.xlabel('Top-k Genes')
        plt.ylabel('Intersection Size')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

    return normalized_aucc

In [10]:
def convert_to_geneid(positions,df):
    result_genes = []
    
    for pos in positions:
        if isinstance(pos,list):
            pos = pos[0]
        pos = pos.split('_')
        if len(pos) == 3:
            chrom, start, end = pos
        elif len(pos) == 2:
            chrom, start = pos
        else:
            pass
        
        start = int(start);
        # 找到相同染色体的基因
        chr_genes = df[df['chrom'] == chrom].copy()
        
        if len(chr_genes) == 0:
            result_genes.append("No gene found")
            continue
            
        # 找到包含start位置的基因
        containing_genes = chr_genes[
            (chr_genes['start'] <= start) & (chr_genes['end'] >= start)
        ].copy()
        
        if len(containing_genes) == 0:
            result_genes.append("No gene found")
            continue
            
        # 如果有protein_coding的基因，优先选择protein_coding
        protein_coding = containing_genes[containing_genes['gene_type'] == 'protein_coding']
        
        if len(protein_coding) > 0:
            # 如果有多个protein_coding基因，选择第一个
            best_gene = protein_coding.iloc[0]
        else:
            # 如果没有protein_coding，选择第一个基因
            best_gene = containing_genes.iloc[0]
        
        result_genes.append(best_gene['gene_id'])
    
    return result_genes

In [11]:
def get_anno_df(f):
    df = pd.read_csv(f, sep='\t', usecols=['gene_id', 'gene_type', 'position'])
    df[['chrom', 'coords']] = df['position'].str.split(':', expand=True)
    df[['start', 'end']] = df['coords'].str.split('-', expand=True).astype(int)
    return df

In [12]:
sc_files = {'data2_24h':'/disk1/2.methods_AS/3.scquint/2.stemCell_GSE52529/Myoblast_Cell_T0_vs_Myoblast_Cell_T24_intron_groups.csv',
           'data2_48h':'/disk1/2.methods_AS/3.scquint/2.stemCell_GSE52529/Myoblast_Cell_T0_vs_Myoblast_Cell_T48_intron_groups.csv',
           'data2_72h':'/disk1/2.methods_AS/3.scquint/2.stemCell_GSE52529/Myoblast_Cell_T0_vs_Myoblast_Cell_T72_intron_groups.csv',
           'data8_ipsc':'/disk1/2.methods_AS/3.scquint/8.iPSC_GSE85908/Motor_neurons_vs_Induced_pluripotent_cells.csv',
           'data9_a2i':'/disk1/2.methods_AS/3.scquint/9.CSC_E-MTAB-2600/serum_vs_a2i.csv'}

In [13]:
rmats_files = {'data2_24h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_24hrs/SE.MATS.JCEC.txt',
               'data2_48h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_48hrs/SE.MATS.JCEC.txt',
               'data2_72h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_72hrs/SE.MATS.JCEC.txt',
              'data8_ipsc':'/disk4/humanData/3.iPSC_GSE85908/bulk_spl/Induced_pluripotent_cells/SE.MATS.JCEC.txt',
              'data9_a2i':'/disk4/mouseData/CSC_E-MTAB-2600/bulk_spl/serum_bulk/SE.MATS.JCEC.txt'}

In [14]:
leaf_files = {'data2_24h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_24hrs_groups_cluster_significance.txt',
             'data2_48h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_48hrs_groups_cluster_significance.txt',
             'data2_72h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_72hrs_groups_cluster_significance.txt',
             'data8_ipsc':'/disk4/humanData/3.iPSC_GSE85908/bulk_spl/Induced_pluripotent_cells_groups_cluster_significance.txt',
             'data9_a2i':'/disk4/mouseData/CSC_E-MTAB-2600/bulk_spl/a2i_bulk_groups_cluster_significance.txt'}

In [15]:
hg_anno_df = get_anno_df(f='/disk1/0.Genome/gencode.v31/GenomeAnnotation.txt')
mm_anno_df = get_anno_df(f='/disk1/0.Genome/gencode.vM25/GenomeAnnotation.txt')

In [21]:
result = {}
gene_result = {}
k = 200

for name in sc_files:
    print(name)
    result[name] = {};gene_result[name] = {}
    sc_f = sc_files[name]; rmats_f = rmats_files[name]; leaf_f = leaf_files[name]
    sc_pval = get_sc(sc_f)
    rmats_pval = get_rmats(rmats_f)
    leaf_pval = get_leaf(leaf_f)
    tool = 'rmats'
    aucc_score = compute_aucc(rmats_pval['5end_3end'].to_list(),sc_pval['pos'].to_list(), k=k, tool=tool)
    result[name][tool] = aucc_score
    tool = 'leaf'
    aucc_score = compute_aucc(leaf_pval['5end_3end'].to_list(),sc_pval['pos'].to_list(), k=k, tool=tool)
    result[name][tool] = aucc_score
    #################gene level##################
    tool = 'rmats'
    aucc_score = compute_aucc_gene(name,rmats_pval['5end_3end'].to_list(),sc_pval['pos'].to_list(), k=k,tool=tool)
    gene_result[name][tool] = aucc_score
    tool = 'leaf'
    aucc_score = compute_aucc_gene(name,leaf_pval['5end_3end'].to_list(),sc_pval['pos'].to_list(), k=k,tool=tool)
    gene_result[name][tool] = aucc_score

data2_24h
data2_48h
data2_72h
data8_ipsc
data9_a2i


In [17]:
gene_result

{'data2_24h': {'rmats': 0.17405940594059405, 'leaf': 0.15445544554455445},
 'data2_48h': {'rmats': 0.2091089108910891, 'leaf': 0.17544554455445543},
 'data2_72h': {'rmats': 0.2857425742574257, 'leaf': 0.2801980198019802},
 'data8_ipsc': {'rmats': 0.1782178217821782, 'leaf': 0.2104950495049505},
 'data9_a2i': {'rmats': 0.22851485148514852, 'leaf': 0.16455445544554456}}

In [18]:
result

{'data2_24h': {'rmats': 0.11108910891089109, 'leaf': 0.11445544554455446},
 'data2_48h': {'rmats': 0.09702970297029703, 'leaf': 0.10435643564356435},
 'data2_72h': {'rmats': 0.14118811881188117, 'leaf': 0.17207920792079207},
 'data8_ipsc': {'rmats': 0.11762376237623762, 'leaf': 0.17603960396039603},
 'data9_a2i': {'rmats': 0.21841584158415842, 'leaf': 0.14237623762376236}}

In [22]:
df = pd.DataFrame.from_dict(result, orient='index')
df.to_csv('scQuint_AUCC.{}.csv'.format(k))

In [23]:
df = pd.DataFrame.from_dict(gene_result,orient='index')
df.to_csv('scQuint_AUCC.gene.{}.csv'.format(k))